<a href="https://colab.research.google.com/github/gracexwho/drug-drug-interactions/blob/master/Encoder_Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from random import choice
import urllib.request  # the lib that handles the url stuff
import time


url = "https://raw.githubusercontent.com/gracexwho/drug-drug-interactions/master/ChCh-Miner_durgbank-chem-chem.tsv"
url_data = urllib.request.urlopen(url) 

G = nx.read_edgelist(url_data)

print(G.number_of_nodes())
print(G.number_of_edges())

1514
48514


In [2]:
# Get training set validation set

num_train = 100
num_val = 20

graph_nodes = list(G.nodes())
training_nodes = torch.zeros([1, num_train], dtype=torch.float)
validation_nodes = torch.zeros([1, num_val], dtype=torch.float)
node_dict = {}
# encode Nodes as numbers

index = 0
for node in list(G.nodes()):
  node_dict[index] = node
  index += 1





for i in range(num_train):
  training_nodes[0][i] = choice(range(len(node_dict)))
  if i < num_val:
    validation_nodes[0][i] = choice(range(len(node_dict)))
   

print(training_nodes.shape)
print(validation_nodes.shape)


def Jaccard(T):
  JC = {}
  listNodes = list(T.nodes)
  for u in listNodes:
    for v in listNodes:
      common = len(list(nx.common_neighbors(T, u, v)))
      union = T.degree[u] + T.degree[v] - common
      if union == 0:
        JC[(u,v)] = 0
      else:
        JC[(u,v)] = (common/union)
    
  return JC


start_time = time.time()

similarity = Jaccard(G)

print("--- %s minutes ---" % ((time.time() - start_time)//60))

# 5:51:99 for 500 nodes in training_nodes

torch.Size([1, 100])
torch.Size([1, 20])
--- 6.0 minutes ---


In [0]:
# Now creating the mapping
# map nodes -> R^d
# decoder in training
# lesson 7: Encoders-Decoders

class Encoder(nn.Module):
  # should return VECTORS for each node
  def __init__(self, num_train):
    super(Encoder, self).__init__()
    #self.fc1 = nn.Linear(1, 256)
    #self.fc2 = nn.Linear(256, 64)
    #self.fc3 = nn.Linear(64, 2)
    
    #self.dropout = nn.Dropout(p=0.2)
    # one layer, return embeds(node_ids) which is a long tensor
    #learnrate might be too big
    self.embed = nn.Embedding(num_train, 256)
    
    
    
  def forward(self, x):
    x = self.embed(x)
    return x

# embeds.weight = embeds.weight/np.sqrt(mbed_dim)
  # Loss function can't be a float, it should be a tensor
  # and also DON'T unwrap a tensor at any point, that gets rid of grad
  # Keep it in tensor operators: maybe change node_dict into a tensor
  
def LossFunction(nodes, similarity):
  # decoder represents ALREADY DECODED results
  # similarity represents evaluated similarities
  # decode is a tuple (u,v) of integers
  u, v = nodes
  a = model(torch.tensor([[u]]))
  b = model(torch.tensor([[v]]))
  
  decoded = Decoder(a,b)
  sim = torch.tensor(similarity[(node_dict[u.item()], node_dict[v.item()])])
  loss = (decoded) - sim       ##
  loss = torch.pow(loss, 2)
  return loss
  
  
def Decoder(v1, v2):
  v1 = torch.squeeze(v1)
  v2 = torch.squeeze(v2)
  # calculate inner product
  return torch.dot(v1, v2)
        


  


In [0]:
## Train


model = Encoder()
optimizer = optim.SGD(model.parameters(), lr=0.1)

epochs = 5

 # have to somehow get vectors representing each node
 # pass in a SINGLE NODE to encoder and get back a vector
 # Comet.ml
  

encoded = {}

for e in range(epochs):
  total_lf = 0
  running_loss = 0
  for u in training_nodes[0]:
      #print(encoded[u])
      
    for v in training_nodes[0]:
      optimizer.zero_grad()
          
      total_lf = LossFunction((u, v), similarity)
      running_loss += total_lf
      total_lf.backward()        #retain_graph=True
      optimizer.step()
      #print(total_lf)

  print(f"Training loss: {running_Loss/num_train}")
      
     # loss with AUC
      # Friday 10:30-11 Meeting
      
        

print(model.parameters())
